In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import sys
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
os.chdir('../..')
os.getcwd()

'c:\\Users\\user\\Downloads\\ten_academy\\week1\\Week1-10Academy-User-Analysis'

In [3]:
from src.plots import Plot

In [4]:
df = pd.read_csv('data/tel_dump.csv')

In [5]:
plot = Plot()

In [6]:
df.columns

Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [8]:
df.isna().sum()

Bearer Id                                      991
Start                                            1
Start ms                                         1
End                                              1
End ms                                           1
Dur. (ms)                                        1
IMSI                                           570
MSISDN/Number                                 1066
IMEI                                           572
Last Location Name                            1153
Avg RTT DL (ms)                              27829
Avg RTT UL (ms)                              27812
Avg Bearer TP DL (kbps)                          1
Avg Bearer TP UL (kbps)                          1
TCP DL Retrans. Vol (Bytes)                  88146
TCP UL Retrans. Vol (Bytes)                  96649
DL TP < 50 Kbps (%)                            754
50 Kbps < DL TP < 250 Kbps (%)                 754
250 Kbps < DL TP < 1 Mbps (%)                  754
DL TP > 1 Mbps (%)             

In [9]:
df = df.query("`Handset Manufacturer` != 'undefined' and `Handset Type` != 'undefined'")[df.columns]

In [10]:
# Data Cleaning: Handling missing values for relevant columns

# Fill missing numerical values with the mean of the respective column
df['TCP DL Retrans. Vol (Bytes)'] = df['TCP DL Retrans. Vol (Bytes)'].fillna(df['TCP DL Retrans. Vol (Bytes)'].mean())
df['TCP UL Retrans. Vol (Bytes)'] = df['TCP UL Retrans. Vol (Bytes)'].fillna(df['TCP UL Retrans. Vol (Bytes)'].mean())
df['Avg RTT DL (ms)'] = df['Avg RTT DL (ms)'].fillna(df['Avg RTT DL (ms)'].mean())
df['Avg RTT UL (ms)'] = df['Avg RTT UL (ms)'].fillna(df['Avg RTT UL (ms)'].mean())
df['Avg Bearer TP DL (kbps)'] = df['Avg Bearer TP DL (kbps)'].fillna(df['Avg Bearer TP DL (kbps)'].mean())
df['Avg Bearer TP UL (kbps)'] = df['Avg Bearer TP UL (kbps)'].fillna(df['Avg Bearer TP UL (kbps)'].mean())



# Fill missing categorical values with the mode
df['Handset Type'] = df['Handset Type'].fillna(df['Handset Type'].mode()[0])
df['MSISDN/Number'] = df['MSISDN/Number'].fillna(df['MSISDN/Number'].mode()[0])

# Confirm that missing values have been addressed in these columns
cleaned_data_info = df[['Avg Bearer TP UL (kbps)', 'Avg Bearer TP DL (kbps)' ,'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Handset Type', 'MSISDN/Number']].isnull().sum()

cleaned_data_info  # Output to confirm the missing values have been handled


Avg Bearer TP UL (kbps)        0
Avg Bearer TP DL (kbps)        0
TCP DL Retrans. Vol (Bytes)    0
TCP UL Retrans. Vol (Bytes)    0
Avg RTT DL (ms)                0
Avg RTT UL (ms)                0
Handset Type                   0
MSISDN/Number                  0
dtype: int64

In [ ]:
# Assuming 'data' contains all necessary columns including 'Session Duration (ms)' and 'Total Traffic (Bytes)'
# and that these columns are correctly named in the dataset

# Calculate engagement metrics
user_engagement = cleaned_data_info.groupby('MSISDN/Number').agg({
    'Bearer Id': 'count',                          # Sessions frequency
    'Dur. (ms)': 'mean',                          # Average session duration
    'Total DL (Bytes)': 'sum',                    # Total Downlink Traffic for all sessions
    'Total UL (Bytes)': 'sum'                     # Total Uplink Traffic for all sessions
})

# Summing uplink and downlink to get total traffic
user_engagement['Total Traffic (Bytes)'] = user_engagement['Total DL (Bytes)'] + user_engagement['Total UL (Bytes)']
user_engagement = user_engagement.drop(['Total DL (Bytes)', 'Total UL (Bytes)'], axis=1)
user_engagement.rename(columns={'Bearer Id': 'Sessions Frequency', 'Dur. (ms)': 'Average Session Duration (ms)'}, inplace=True)

# Showing the head of the engagement data for verification
user_engagement.head()
